**Important: All references to project, COS bucket, and API keys must be replaced before running this notebook in your project. Search this notebook for "replace"**

### Scoring a Custom Model
We use a generic term *custom model* to describe a model that was saved into a project as a file using a Python API and not WML API. 

While we recommend that models are saved using the WML API and deployed as *models* in Deployment Spaces, there may be cases when the models may need to be saved as files. Three typical reasons for using this approach are:
1. A model was trained outside of Watson Studio, but it needs to be deployed in Deployment Spaces
2. Framework used by the model is not supported by default in Watson Studio, that's why WML API can't be used. You can find the list of supported frameworks in documentation: https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=functions-supported-deployment-frameworks
3. A customer has an established deployment process which works with exported model files. 

*In this notebook we demontrate how to load the model from the project (the model is available in Data Assets). You can also load the model file from any storage type - Storage Volume (shared file system), Git, Object Storage, etc.*

### Step 1: Load the model from Data Assets

In [ ]:
# Get the file - the file name was specified in the MortgageDefault-sklearn notebook
buffer = project.get_file("mortgage_default_model_custom")
type(file)

In [ ]:
import pickle

# load the model from buffer
loaded_model = pickle.load(buffer)

# Verify that the model was serialized into the right object
type(loaded_model)

### Step 2: Load and preview data that will be scored

In [ ]:
# Instead of repacing the COS API key and bucket, you can regenerate code and replace the pandas data frame name. Change it to 
#scoring_input_data

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_1e498447d5f74cd6b90b92b35bb6514e = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_1e498447d5f74cd6b90b92b35bb6514e = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_1e498447d5f74cd6b90b92b35bb6514e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='replace-with-your-cos-api-ky',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_1e498447d5f74cd6b90b92b35bb6514e)

body = client_1e498447d5f74cd6b90b92b35bb6514e.get_object(Bucket='replace-with-your-cos-bucket')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

scoring_input_data = pd.read_csv(body)
scoring_input_data.head()


### Step 3: Score data

In [ ]:
# Get scoring results and probability of churn (prediciton of T=True)
scoring_results = loaded_model.predict(scoring_input_data)
scoring_prob_T = loaded_model.predict_proba(scoring_input_data)[:, 1]

In [ ]:
# Scoring results are in an array format - the order is the same as the order of records in input, 
# and that's why we can append the input data with results based on order
print(scoring_results)

scoring_input_data['prediction']=scoring_results
scoring_input_data['probability_T'] = scoring_prob_T

In [ ]:
scoring_output_data=scoring_input_data
# Display the data frame that we will write to a file
scoring_output_data

### Step 4: Save scoring results to Data Assets
In this example we save scoring results to Data Assets. You can also save scoring results to other data sources - Storage Volume (shared file system), Git, Object Storage, databases, Hadoop, etc. 

In [ ]:
project.save_data("ScoringOutput_Custom_Model.csv", scoring_output_data.to_csv(index=False))

**Author:**  Elena Lowery and Catherine Cao <br/>
**Date:**  December 2021